In [1]:
###############################################################################################################################
#
#   Copyright © 2022 Center for Agricultural Systems Biology
#   Authorship and citation: Ruengsrichaiya B., Nukoolkit C., Kalapanulak S. and Saithong T., 
#   (202x) Plant-DTI: Extending the landscape of TF protein and DNA interaction in plants by a machine learning-based approach. 
#   xxxxx., xx, xxx. (in preperation). 
#   Contact: bhukrit.r@mail.kmutt.ac.th
#
###############################################################################################################################
#   
#   Code for Plant-DTI model prediction: 
#   Classes labeled  are 1 (interacted) and 0 (not interacted)
#   Random within models (RW) are avaiable for TFBS length range from 7-15 bp.
#   Random pairs models (RP) are avaiable for TFBS length range from 7-14 bp.
#
###############################################################################################################################

#Import all required library
import pandas as pd  
import numpy as np  
import pickle
from Bio.Seq import Seq
from sklearn.ensemble import RandomForestClassifier #sklearn version 0.23.1 with python 3.6 or newer



###################################################################################################################
#
#                                            Feature Representation 
#
###################################################################################################################

#Amino acid mode of preference to DNA 

def aa_mode_of_preference_to_dna(DBD):
    #create list for collect data
    H_bond=[]
    vanderWaal=[]
    no_base_contact=[]
    
    #count aa sequence and length of aa for each line
    for i in range(len(DBD)):
        seq = DBD[i]
        seq_len = len(DBD[i])
        
        R = seq.count('R')
        K = seq.count('K')
        H = seq.count('H')
        S = seq.count('S')
        N = seq.count('N')
        Q = seq.count('Q')
        D = seq.count('D')
        E = seq.count('E')
        F = seq.count('F')
        P = seq.count('P')
        T = seq.count('T')
        G = seq.count('G')
        A = seq.count('A')
        V = seq.count('V')
        L = seq.count('L')
        I = seq.count('I')
        Y = seq.count('Y')
        C = seq.count('C')
        M = seq.count('M')
        W = seq.count('W')
        count_H_bond = (R+K+H+S+N+Q+D+E)/seq_len
        H_bond.append(count_H_bond)
        count_vanderWaal = (F+P+T+G+A+V+L+I+Y)/seq_len
        vanderWaal.append(count_vanderWaal)
        count_no_base=(C+M+W)/seq_len
        no_base_contact.append(count_no_base)
        
    #create df for H-bond
    H_bond_df = pd.DataFrame(H_bond)
    H_bond_df.columns= ['H-bond']
    
    #create df for van der Waal
    vanderWaal_df = pd.DataFrame(vanderWaal)
    vanderWaal_df.columns = ['vanderWaal']
    
    #create df for no base contact
    no_basecontact_df=pd.DataFrame(no_base_contact)
    no_basecontact_df.columns = ['no_basecontact']
    
    related_interaction_df= pd.concat([H_bond_df,vanderWaal_df,no_basecontact_df],axis=1)
    return related_interaction_df




# TFBS base preference to DBD type 

def TFBS_preference_to_DBD_type(interaction_df):
    
    TFBS_df=interaction_df[['pfam_name','TFBS_seq']]
    length = len(TFBS_df['TFBS_seq'][0]) #get length from TFBS_seq in row 1 
    seq_ls=['A','T','C','G']
    score_dict={}
    
    pwm_df = pd.read_csv('Model/TFBS_base_preference/TFBS_base_preference_len' + str(length) + '.csv')
    pwm_df.set_index('pfam_Name',inplace=True)

    for base in seq_ls:
        for i in range(length):
            score_dict[base+str(i+1)]=[]

    score_df=pd.DataFrame()
    
    for i in range(len(TFBS_df)):
        for k in range(len(TFBS_df.TFBS_seq[i])):
            nucleotid_and_pos = TFBS_df.TFBS_seq[i][k] + str(k+1)
            nucleotid_and_pos_freq = pwm_df.loc[TFBS_df.pfam_name[i]][nucleotid_and_pos]
            score_dict[nucleotid_and_pos].append(nucleotid_and_pos_freq)
        for key in score_dict.keys():
            if len(score_dict[key]) != i+1:
                score_dict[key].append('')
    new_df=pd.DataFrame(score_dict)
    new_df=new_df.replace('',int(0))
    return new_df

#Sliding window
#Code modified from: https://stackoverflow.com/questions/53681130/sliding-window-and-recognizing-specific-characters-in-a-list
def get_chunks(dna, window_size, stride=1):
    for i in range(0, len(dna) - window_size + 1, stride):
        chunk = dna[i:i + window_size]
        start_pos= i + 1
        end_pos= i + window_size
        strand = '+'
        assert len(chunk) == window_size
        yield [chunk,start_pos,end_pos,strand]
    

In [2]:
#######################################################################################################
#Set INPUT DBD-TFBS interaction probability
# DBD-TFBS interaction probability can use from > 0.5 to 1, 
#The DBD-TFBS interaction probability closely to 1 is more confidence predicted DBD-TFBS interaction
#Defalut of DBD-TFBS interaction probability is > 0.7
prob_thredshold = 0.7

#Read example file input for prediction
interaction_df = pd.read_csv('example_input.csv')

#######################################################################################################

#Read file for checking model aviability for each DBD type: link_DBD_TFBS_len_model
link_model_df = pd.read_csv('Model/PlantDTI_models_availability.csv')

#Mapping input to model aviability
link_model_DBD_df = pd.merge(link_model_df, interaction_df, on=['pfam_name', 'len_TFBS'], how='inner')

#Create dataframe for collecting prediction results
predicted_result_df = pd.DataFrame()

for i in range(len(link_model_DBD_df)):
    
    interaction_row_df = link_model_DBD_df[i:i+1].reset_index(drop=True)
    len_model = interaction_row_df['len_TFBS'][0]
    model_type = interaction_row_df['generate_data'][0]
    
    #Feature representation
    #DBD feature
    DBD_df = aa_mode_of_preference_to_dna([link_model_DBD_df['DBD_seq'][i]])
    
    #TFBS feature
    TFBS_df = TFBS_preference_to_DBD_type(interaction_row_df)
    
    #Create feature_vector by combine TFBS and DBD features
    feature_vector_df= pd.concat([TFBS_df, DBD_df],axis=1)

    #Prediction in particular model length and type
    model_name = 'Model/' + 'len ' +  str(len_model) + ' ' + model_type +'.sav'
    loaded_model = pickle.load(open(model_name, 'rb'))


    #Prediction result from model
    predict_prob=loaded_model.predict_proba(feature_vector_df)[:,1] #collect prob of predicted as interact
    predict_prob_df = pd.DataFrame(predict_prob,columns=['predict_prob_result'])
    
    result_df = pd.DataFrame()
    result_df = pd.concat([result_df,interaction_row_df,predict_prob_df],axis=1) #concat predict result to  retrived sequence
    predicted_result_df = pd.concat([predicted_result_df, result_df]) 
    
    # Retrived row above prob_thredshold which predicted as interact
    predicted_result_df = predicted_result_df[predicted_result_df['predict_prob_result']>prob_thredshold]

predicted_result_df.to_csv('Result/example_result.csv', index=False)
    

In [3]:
predicted_result_df

,len_TFBS,generate_data,pfam_name,Protein_ID,DBD_seq,TFBS_seq,predict_prob_result
0,8,RW,AP2,Manes.15G009900.1.p,YRGIRQRPWGKWAAEIRDPHKGVRVWLGTYNTAEEAARAYDEAAKRIRG,ATGCCGAG,0.72
0,9,RP,AP2,Manes.15G009900.1.p,YRGIRQRPWGKWAAEIRDPHKGVRVWLGTYNTAEEAARAYDEAAKRIRG,ATGCCGAGA,0.95
0,9,RW,AP2,Manes.15G009900.1.p,YRGIRQRPWGKWAAEIRDPHKGVRVWLGTYNTAEEAARAYDEAAKRIRG,ATGCCGAGA,0.99
0,10,RP,GATA,Manes.07G041300.1.p,CTHCGTSSKSTPMMRRGPAGPRTLCNACGLKWANKG,AATCATGGGA,0.94
0,11,RP,zf-Dof,Manes.01G244000.1.p,PQEQLNCPRCNSTNTKFCYYNNYSLTQPRYFCKTCRRYWTEGGSLR...,TGGAAAAGTTA,1.00
0,11,RW,zf-Dof,Manes.01G244000.1.p,PQEQLNCPRCNSTNTKFCYYNNYSLTQPRYFCKTCRRYWTEGGSLR...,TGGAAAAGTTA,1.00
